# Import
* !pip show pandas
* !pip install jupyter ipywidgets
* !pip install --upgrade pip
* !pip install tabulate matplotlib pandas

In [91]:
import io
import ipywidgets as widgets
from ipywidgets import Output
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import re
out = Output()
out_hist = Output()

In [92]:
from biblioteca import listar_pastas, list_alphas_ordered
# import biblioteca
path_to_project = '/home/wsl/git/Wf-Security/heuristic_and_exact_solutions'
output_folder = 'output'
output_file_name = 'wf_security_executions.csv'

In [93]:
outputs = listar_pastas(f'{path_to_project}/{output_folder}')
executions = None
instances = ['All']
scenarios = []
alphas = []

In [94]:
dd_outputs = widgets.Dropdown(options=outputs, description='Output')
dd_instances = widgets.Dropdown(options=instances, description='Instance')
dd_scenarios = widgets.Dropdown(options=scenarios, description='Scenario')
dd_alphas = widgets.Dropdown(options=alphas, description='Alphas')
hbox1 = widgets.HBox([dd_outputs])
hbox2 = widgets.HBox([dd_instances, dd_scenarios, dd_alphas])
vbox = widgets.VBox([hbox1, hbox2])
tab = widgets.Tab()
tab.children = [out, out_hist]
tab.set_title(0, 'Table')
tab.set_title(1, 'Histogram')

In [95]:
def background_color(index, num_columns):
    quotient = int(index) // 2
    if quotient % 2 == 0:  # Se o índice for par
        return ['background-color: #f2f2f2'] * num_columns  # Define light gray background
    else:
        return ['background-color: #ffffff'] * num_columns  # Define white background
        
def hilite_min(s):
    is_minimo = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_minimo]

def query_executions(instance, scenario, alphas):
    instance_filter = executions if instance == 'All' else executions[executions.instance == instance]
    scenarios_filter = instance_filter if scenario == 'All' else instance_filter[(instance_filter.scenario == scenario)]
#                                                                                  & (instance_filter.algorithm == 'cplex')]
    if alphas == 'All':
        alphas_filter = scenarios_filter
    else:
        alphas_float = [float(alpha) for alpha in alphas]
        alphas_filter = scenarios_filter[(scenarios_filter.alpha_time == alphas_float[0])
                                         & (scenarios_filter.alpha_cost == alphas_float[1])
                                         & (scenarios_filter.alpha_security == alphas_float[2])
                                         & (scenarios_filter.alpha_restricted_candidate_list == alphas_float[3])]
    alias = {"num_iterations": "n_iter", "object_function": "O.F.", "makespan": "mspn", "cost": "cost", "security": "security",
             "time": "time", "best_solution_iteration": "bsi", "best_solution_time": "bst", "lsn_time_1": "lsnt_1",
             "lsn_noi_1": "lsni_1", "lsn_time_2": "lsnt_2", "lsn_noi_2": "lsni_2", "lsn_time_3": "lsnt_3", "lsn_noi_3": "lsni_3",
             'alpha_restricted_candidate_list': 'RCL', 'num_buckets': 'n_bckts', 'alpha_time': 'a_t', 'alpha_cost': 'a_c', 
             'alpha_security': 'a_s', 'algorithm': 'lgrthm', 'scenario': '#'}
    alphas_filter = alphas_filter.rename(columns=alias)
    desired_columns = ['O.F.',  'mspn',  'cost',  'security',  'time', 'n_iter',  'bsi', 'bst']
    for column in desired_columns:
        alphas_filter.loc[alphas_filter[column] < 0, column] = None
    grouped_executions = alphas_filter.groupby(['lgrthm', 'instance', '#', 'a_t', 'a_c', 'a_s', 'RCL', 'n_bckts'])[desired_columns].mean()
    grouped_executions = grouped_executions.rename(columns=alias)
    grouped_executions_sorted = grouped_executions.sort_values(by=['instance', 'lgrthm'], ascending=True)
    out.clear_output()
    with out:
        if instance == 'All':
            display(grouped_executions_sorted.style.apply(lambda x: background_color(grouped_executions_sorted.index.get_loc(x.name), 
                                                                                     len(grouped_executions_sorted.columns)), 
                                                          axis=1))
        else:
            display(grouped_executions_sorted.style.apply(hilite_min))
    out_hist.clear_output()
    with out_hist:
        pass
#         data = {
#             ('grasp', 'inst1'): [0.4],
#             ('grch', 'inst1'): [0.5],
#             ('grasp', 'inst2'): [0.3],
#             ('grch', 'inst2'): [0.6]
#         }
#         df = pd.DataFrame(data)
        
#         # Renomeando as colunas do DataFrame
#         df.columns = ['_'.join(col).strip() for col in df.columns.values]
        
#         # Plotando o gráfico de barras
#         df.plot(kind='bar')
#         plt.xlabel('Instância')
#         plt.ylabel('Valor da função objetivo (O.F.)')
#         plt.title('Comparação de algoritmos por instância')
#         plt.legend(title='Algoritmo')
#         plt.show()
#         # data = {
#         #     ('grasp', 'inst1'): [0.4],
#         #     ('grch', 'inst1'): [0.5],
#         #     ('grasp', 'inst2'): [0.3],
#         #     ('grch', 'inst2'): [0.6]
#         # }
#         # df = pd.DataFrame(data)

#         grouped_executions_dropped_columns = grouped_executions_sorted.drop(columns=['mspn', 'cost', 'security', 'time', 'n_iter', 'bsi', 'bst'])
        
#         # Renomeando as colunas do DataFrame
#         grouped_executions_dropped_columns.columns = ['_'.join(col).strip() for col in grouped_executions_dropped_columns.columns.values]
        
#         # Plotando o gráfico de barras
#         grouped_executions_dropped_columns.plot(kind='bar')
#         plt.xlabel('Instância')
#         plt.ylabel('Valor da função objetivo (O.F.)')
#         plt.title('Comparação de algoritmos por instância')
#         plt.legend(title='Algoritmo')
#         plt.show()
        
#         # algorithm = grouped_executions_sorted.lgrthm.unique()
#         # print(algorithm)
#         # instances = grouped_executions_sorted.instance.unique()
#         # print(instances)
#         # grouped_executions_sorted.plot.bar(rot=0)
#         # plt.figure()
#         # grouped_executions_sorted.hist('O.F.')
#         # plt.savefig('histogram.png')  # Salva a figura como 'histogram.png'
#         # plt.close()  # Fecha a figura
#         # # Exibe a figura dentro do widget out_hist
#         # with open('histogram.png', 'rb') as img_file:
#         #     img = img_file.read()
#         #     display(widgets.Image(value=img, format='png'))
        
def fetch_data(output):
    global executions
    executions = pd.read_csv(f'{path_to_project}/{output_folder}/{output}/{output_file_name}', header='infer')
    dd_instances.options = ['All'] + list(executions.instance.unique())
    dd_scenarios.options = list(executions.scenario.unique())
    if dd_scenarios.options:
        dd_scenarios.value = dd_scenarios.options[0]
    dd_alphas.options = list_alphas_ordered(f'{path_to_project}/{output_folder}/{output}/{output_file_name}')
    if dd_alphas.options:
        dd_alphas.value = dd_alphas.options[0]
    query_executions(dd_instances.value, dd_scenarios.value, dd_alphas.value)

fetch_data(dd_outputs.value)

In [96]:
def dd_outputs_observer(bunch):
    fetch_data(bunch.new)
def dd_instances_observer(bunch):
    query_executions(bunch.new, dd_scenarios.value, dd_alphas.value)
def dd_scenarios_observer(bunch):
    query_executions(dd_instances.value, bunch.new, alphas.value)
def dd_alphas_observer(bunch):
    query_executions(dd_instances.value, dd_scenarios.value, bunch.new)

In [97]:
dd_outputs.observe(dd_outputs_observer, names='value')
dd_instances.observe(dd_instances_observer, names='value')
dd_scenarios.observe(dd_scenarios_observer, names='value')
dd_alphas.observe(dd_alphas_observer, names='value')

In [98]:
display(vbox)

In [99]:
display(tab)

In [10]:
# display(out_hist)

In [11]:
import pandas as pd

# Criar um exemplo de DataFrame
df = pd.DataFrame({'A': [11, 22, 33], 'B': [44, 55, 66]})

# Imprimir o DataFrame original
print("DataFrame original:")
print(df)

# Obter o índice numérico das linhas
indice_numerico = df.index

# Imprimir o índice numérico
print("\nÍndice numérico das linhas:")
print(indice_numerico)
print(indice_numerico[2])

DataFrame original:
    A   B
0  11  44
1  22  55
2  33  66

Índice numérico das linhas:
RangeIndex(start=0, stop=3, step=1)
2


In [12]:
import pandas as pd

# Criar um exemplo de DataFrame com MultiIndex
arrays = [
    ['A', 'A', 'B', 'B'],
    [1, 2, 1, 2]
]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
df = pd.DataFrame({'data': [5, 6, 7, 8]}, index=index)

# Obter a posição do índice 'A' na primeira coluna do MultiIndex
posicao = df.index.get_loc(('A', 1))
print("A posição do índice ('A', 1) é:", posicao)
posicao = df.index.get_loc(('A', 2))
print("A posição do índice ('A', 2) é:", posicao)
posicao = df.index.get_loc(('B', 1))
print("A posição do índice ('B', 1) é:", posicao)
posicao = df.index.get_loc(('B', 2))
print("A posição do índice ('B', 2) é:", posicao)
df

A posição do índice ('A', 1) é: 0
A posição do índice ('A', 2) é: 1
A posição do índice ('B', 1) é: 2
A posição do índice ('B', 2) é: 3


data
first second      
A     1          5
      2          6
B     1          7
      2          8